**0) Lý thuyết và nguồn dữ liệu** <br>
Nguồn dữ liệu được lấy từ trang web: https://github.com/stefan-it/nmt-en-vi/tree/master/data

Lý thuyết tham khảo ở các tài liệu:


1.   Cho, K., Van Merriënboer, B., Bahdanau, D., & Bengio, Y. (2014). On the properties of neural machine translation: Encoder-decoder approaches. arXiv preprint arXiv:1409.1259.
2.   Rico Sennrich, Barry Haddow (2016). Linguistic Input Features Improve Neural Machine Translation.



**1) Chuẩn bị thư viện và dữ liệu**

Đầu tiên ta sẽ chuẩn bị một số thư viện

In [0]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
!pip install unidecode
import unidecode
import string
import re
import random
import numpy as np

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

     |████████████████████████████████| 245kB 4.5MB/s 


Chuẩn bị dữ liệu cho chương trình

In [18]:
!git clone https://github.com/nguyenduyhanlam/PTXS-GTNN
!ls

Cloning into 'PTXS-GTNN'...
remote: Enumerating objects: 15, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 15 (delta 0), reused 15 (delta 0), pack-reused 0
Unpacking objects: 100% (15/15), done.
adc.json  Filename.csv	PTXS-GTNN  sample_data	tst2013.en  tst2013.vn


**2) Load dữ liệu** <br>
Ta sẽ tạo class ngôn ngữ dùng để load dữ liệu từ file text. <br>
Class ngôn ngữ gồm có các **thuộc tính** cơ bản sau:
*   **name**: tên của ngôn ngữ (English, tiếng Việt,...)
*   **word2index**: bộ từ điển.
*   **word2count**: bộ đếm từ.
*   **index2word**: bộ lưu vị trí các token.
*   **n_words**: đếm tổng các lượng từ.

Class ngôn ngữ gồm có các **hàm** cơ bản sau:
*   **addSentence**: Có tham số là sentence, dùng để truyền vào 1 câu văn bản. Từ câu văn bản này, ta sẽ tách thành các từ (word). Cuối cùng ta sẽ lưu các từ này vào bộ từ điển bằng hàm addWord.
*   **addWord**: Có tham số là word, dùng để truyền vào 1 từ (word), nếu từ này chưa có trong bộ từ điển thì sẽ được thêm (add) mới vào bộ từ điển. Nếu đã có rồi ta sẽ tăng số lần xuất hiện (số đếm) của từ này thêm 1 đơn vị.








In [0]:
SOS_token = 0 #Start Of Sequence
EOS_token = 1 #End Of Sequence
PAD_token = 2 # Used for padding short sentences

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

Ta cần triển khai thêm một số hàm bổ trợ như:
*   **remove_accent**: dùng để loại bỏ các dấu thanh (sắc, hỏi, huyền, ngã, nặng,...).
*   **normalizeString**: Dùng để chuẩn hóa câu để chạy chương trình. Đầu tiên ta sẽ loại bỏ hết các dấu thanh ra khỏi câu. Sau đó ta sẽ tách các dấu câu (dấu phẩy, dấu chấm, dấu chấm hỏi, dấu chấm thang,...) ra khỏi từ.

Ví dụ ta có câu: "Ăn quả, nhớ kẻ trồng cây.". Sau khi chạy hàm **normalizeString** ta sẽ được kết quả là: "An qua , nho ke trong cay ."



In [11]:
# remove all the accents
def remove_accent(utf8_str):
    return unidecode.unidecode(utf8_str)

# Normalize the string (marks and words are seperated, words don't contain accents,...)
def normalizeString(s):
    # Remove all the accents first.
    s = remove_accent(s)
    # Seperate words and marks by adding spaces between them
    marks = '[.!?,-${}()]'
    r = "(["+"\\".join(marks)+"])"
    s = re.sub(r, r" \1 ", s)
    # replace continuous spaces with a single space
    s = re.sub(r"\s+", r" ", s).strip()
    return s

# Example
ex_s = "Ăn quả, nhớ kẻ trồng cây."
print(normalizeString(ex_s)) # result will be "An qua , nho ke trong cay ."

An qua , nho ke trong cay .


Cuối cùng ta sẽ thực hiện việc load dữ liệu.

In [20]:
def readLangs(lang1, lang2):
    print("Reading lines...")

    # Read the file and split into lines
    lines_language1 = open('tst2013.%s' % lang1, encoding='utf-8').\
        read().strip().split('\n')
    lines_language2 = open('tst2013.%s' % lang2, encoding='utf-8').\
        read().strip().split('\n')

    # Normalize all the lines
    data_language1 = [normalizeString(l) for l in lines_language1]
    data_language2 = [normalizeString(l) for l in lines_language2]

    # Prepare return values
    input_lang = Lang(lang1)
    output_lang = Lang(lang2)
    data = list(zip(data_language1, data_language2))

    return input_lang, output_lang, data

# Test the function
lang1 = "en"
lang2 = "vn"
input_lang, output_lang, data = readLangs(lang1, lang2)
print("Language 1:", input_lang.name)
print("Language 2:", output_lang.name)
print(random.choice(data))

Reading lines...
Language 1: en
Language 2: vn
('Guess what ?', 'Nghi thu xem nhe ?')


Chương trình sẽ chạy nhanh hơn khi chúng ta cắt bộ dữ liệu thành các câu ngắn và đơn giản. Ở đây ta sẽ giới hạn cho 1 câu có tối đa là 10 từ (bao gồm cả dấu chấm câu).


**3) Chuẩn bị dữ liệu** <br>
